Clone github repo to have access to data files.

In [1]:
!git clone https://github.com/Hananxx/SentimentAnalysisPromptExp.git

Cloning into 'SentimentAnalysisPromptExp'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 23 (delta 8), reused 9 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (23/23), 39.42 KiB | 4.38 MiB/s, done.
Resolving deltas: 100% (8/8), done.


### Install and import needed packages

In [2]:
!pip install transformers torch pandas accelerate

In [3]:
import pandas as pd
import json
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import torch

### Set root path for accessing repo files

In [11]:
file_path_prefix = "SentimentAnalysisPromptExp/"

### Load dataset

In [12]:
df = pd.read_csv(file_path_prefix + 'data/app.csv')
print(df.head())  # Inspect the first few rows

                                            sentence  label
0  package file invalid i had my phone on factory...      2
1  iffy nice clean app but sometimes it works and...      2
2                        cool just freezes everytime      2
3  network error! suddenly after downloading an u...      2
4  annoying it let me choose the pictures i want ...      2


### Load prompts

In [13]:
with open(file_path_prefix + 'prompts/zero-shot-prompt-template.json', 'r') as f:
    templates = json.load(f)
print(templates)

{'vicuna-0': "A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions.\nUSER: Please perform Sentiment Classification task. Given the sentence from {}, assign a sentiment label from ['negative', 'neutral', 'positive']. Return label only without any other text.\nASSISTANT: Sure!</s>\nUSER: Sentence: {}\nASSISTANT:", 'vicuna-jira-0': "A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions.\nUSER: Please perform Sentiment Classification task. Given the sentence from {}, assign a sentiment label from ['negative', 'positive']. Return label only without any other text.\nASSISTANT: Sure!</s>\nUSER: Sentence: {}\nASSISTANT:", 'llama2-0': "<s>[INST] <<SYS>>\nA chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's q

### Use Vicuna

In [28]:
# Load tokenizer and model
models = ['lmsys/vicuna-13b-v1.5', 'WizardLM/WizardLM-13B-V1.2', 'meta-llama/Llama-2-13b-chat-hf']
tokenizer = AutoTokenizer.from_pretrained(models[1])

# Create a text generation pipeline
model_pipeline = pipeline(
    'text-generation',
    model=model_name,
    tokenizer=tokenizer,
    dtype=torch.float16,
    device_map='auto',
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0


In [34]:
prompt = templates['vicuna-2']
sentences = df['sentence'].tolist()[0:50] # For test purposes only the first 50 sentences.

for sentence in sentences:
    full_prompt = prompt.format("APP reviews", sentence)
    output = model_pipeline(
          full_prompt,
          max_length=2048,
          temperature=0.7,
          pad_token_id=tokenizer.pad_token_id,
          eos_token_id=tokenizer.eos_token_id,
          return_full_text=False
    )

    response = output[0]['generated_text'].strip()  # Extract just the label.
    print(response)

The sentiment expressed in this sentence is negative.
The sentiment expressed in the given sentence is negative.
The sentiment expressed in this sentence is negative.
The sentiment expressed in this sentence is negative.
The sentiment expressed in this sentence is negative.
The sentiment expressed in the sentence is initially negative, as the user mentions an annoying bug that occurs when using both the powertoggles notification bar widget and home screen widget. However, the user also provides a positive rating for the app, indicating that the bug does not occur anymore and that the app has a good concept overall.
The sentiment expressed in the sentence is negative.
The sentiment expressed in the given sentence is negative.
The sentiment expressed in this sentence is neutral. The writer acknowledges that the app has some flaws, but also notes that it has some positive aspects, such as a good selection of routing options. However, the writer also mentions some negative experiences, suc